In [1]:
import numpy as np
from scipy import stats 
import math
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [2]:
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.orm import Session, sessionmaker

In [3]:
engine = create_engine("postgresql:///music", echo=True)
Session = sessionmaker(bind=engine)
session = Session()
con = engine.connect()

2019-05-08 17:03:59,804 INFO sqlalchemy.engine.base.Engine select version()
2019-05-08 17:03:59,805 INFO sqlalchemy.engine.base.Engine {}
2019-05-08 17:03:59,807 INFO sqlalchemy.engine.base.Engine select current_schema()
2019-05-08 17:03:59,808 INFO sqlalchemy.engine.base.Engine {}
2019-05-08 17:03:59,810 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-05-08 17:03:59,810 INFO sqlalchemy.engine.base.Engine {}
2019-05-08 17:03:59,812 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-05-08 17:03:59,813 INFO sqlalchemy.engine.base.Engine {}
2019-05-08 17:03:59,814 INFO sqlalchemy.engine.base.Engine show standard_conforming_strings
2019-05-08 17:03:59,815 INFO sqlalchemy.engine.base.Engine {}


In [4]:
query = """
    SELECT best_new_music
    FROM reviews
    """
result = con.execute(query).fetchall()
bnms = np.array(result)

2019-05-08 17:03:59,821 INFO sqlalchemy.engine.base.Engine 
    SELECT best_new_music
    FROM reviews
    
2019-05-08 17:03:59,823 INFO sqlalchemy.engine.base.Engine {}


In [5]:
bnm_list = [tup[0] for tup in bnms]

In [6]:
bnm_list[0]

0

In [7]:
mu_pop = np.mean(bnm_list)
mu_pop

0.05126950470287609

In [8]:
query = """
    SELECT genre
    FROM genres
    GROUP BY genre
    """
result = con.execute(query).fetchall()
genres = np.array(result)

2019-05-08 17:04:00,036 INFO sqlalchemy.engine.base.Engine 
    SELECT genre
    FROM genres
    GROUP BY genre
    
2019-05-08 17:04:00,037 INFO sqlalchemy.engine.base.Engine {}


In [9]:
genre_list = [tup[0] for tup in genres] 

In [10]:
genre_list

['global',
 'experimental',
 'metal',
 None,
 'electronic',
 'folk/country',
 'pop/r&b',
 'jazz',
 'rap',
 'rock']

In [11]:
genre_list.remove(None)

In [12]:
genre_list

['global',
 'experimental',
 'metal',
 'electronic',
 'folk/country',
 'pop/r&b',
 'jazz',
 'rap',
 'rock']

In [13]:
bnm_by_genre = {}
for genre in genre_list:
    query = f"""
    SELECT best_new_music
    FROM reviews
    JOIN genres
    USING (reviewid)
    WHERE genre = '{genre}'
    """
    result = con.execute(query).fetchall() 
    bnm_list = [tup[0] for tup in result]
    bnm_by_genre[genre]=bnm_list

2019-05-08 17:04:00,068 INFO sqlalchemy.engine.base.Engine 
    SELECT best_new_music
    FROM reviews
    JOIN genres
    USING (reviewid)
    WHERE genre = 'global'
    
2019-05-08 17:04:00,070 INFO sqlalchemy.engine.base.Engine {}
2019-05-08 17:04:00,075 INFO sqlalchemy.engine.base.Engine 
    SELECT best_new_music
    FROM reviews
    JOIN genres
    USING (reviewid)
    WHERE genre = 'experimental'
    
2019-05-08 17:04:00,076 INFO sqlalchemy.engine.base.Engine {}
2019-05-08 17:04:00,082 INFO sqlalchemy.engine.base.Engine 
    SELECT best_new_music
    FROM reviews
    JOIN genres
    USING (reviewid)
    WHERE genre = 'metal'
    
2019-05-08 17:04:00,083 INFO sqlalchemy.engine.base.Engine {}
2019-05-08 17:04:00,089 INFO sqlalchemy.engine.base.Engine 
    SELECT best_new_music
    FROM reviews
    JOIN genres
    USING (reviewid)
    WHERE genre = 'electronic'
    
2019-05-08 17:04:00,090 INFO sqlalchemy.engine.base.Engine {}
2019-05-08 17:04:00,098 INFO sqlalchemy.engine.base.Eng

In [14]:
alpha = 0.1
bonferroni_alpha = alpha / len(genre_list)

In [15]:
tests_by_genre = {}
for genre in bnm_by_genre:
    bnms = bnm_by_genre[genre]
    t_val, p_val = stats.ttest_1samp(bnms, mu_pop)
    tests_by_genre[genre] = {
        't_val' : t_val,
        't_crit' : stats.t.ppf(1-bonferroni_alpha, len(bnms) -1),
        'p_val' : p_val,
    }

In [16]:
tests_by_genre['global']

{'t_val': -0.7566877930227294,
 't_crit': 2.3029922469382376,
 'p_val': 0.45005391992707033}

In [36]:
test_results_by_genre = {}
for genre in tests_by_genre:
    t_val = tests_by_genre[genre]['t_val']
    t_crit = tests_by_genre[genre]['t_crit']
    p_val = tests_by_genre[genre]['p_val']
    if np.abs(t_val) > np.abs(t_crit) and np.abs(p_val) < bonferroni_alpha:
        test_results_by_genre[genre] = f'This indicates ' \
        f'that the alternative hypothesis is accepted with a p-val of {p_val}.'
    else: 
        test_results_by_genre[genre] = 'We fail to reject the null hypothesis.'
        

In [37]:
test_results_by_genre

{'global': 'We fail to reject the null hypothesis.',
 'experimental': 'This indicates that the alternative hypothesis is accepted with a p-val of 0.006737474504092826.',
 'metal': 'This indicates that the alternative hypothesis is accepted with a p-val of 0.00031043031912163075.',
 'electronic': 'We fail to reject the null hypothesis.',
 'folk/country': 'We fail to reject the null hypothesis.',
 'pop/r&b': 'We fail to reject the null hypothesis.',
 'jazz': 'We fail to reject the null hypothesis.',
 'rap': 'We fail to reject the null hypothesis.',
 'rock': 'We fail to reject the null hypothesis.'}

In [38]:
tests_by_genre

{'global': {'t_val': -0.7566877930227294,
  't_crit': 2.3029922469382376,
  'p_val': 0.45005391992707033},
 'experimental': {'t_val': 2.712667340358848,
  't_crit': 2.2885122631793666,
  'p_val': 0.006737474504092826},
 'metal': {'t_val': -3.6211575950131696,
  't_crit': 2.290690243733693,
  'p_val': 0.00031043031912163075},
 'electronic': {'t_val': -0.870763140434117,
  't_crit': 2.2874675746937077,
  'p_val': 0.3839374628045431},
 'folk/country': {'t_val': -1.593193149477528,
  't_crit': 2.2917644970463025,
  'p_val': 0.1115788415997883},
 'pop/r&b': {'t_val': 1.294829883663111,
  't_crit': 2.289038550669228,
  'p_val': 0.1955877965837994},
 'jazz': {'t_val': -1.3103617221416028,
  't_crit': 2.294779786135665,
  'p_val': 0.19076654702094978},
 'rap': {'t_val': -0.10725778282023768,
  't_crit': 2.288835335272128,
  'p_val': 0.9145982851368573},
 'rock': {'t_val': 2.0618796291729073,
  't_crit': 2.2869252842832197,
  'p_val': 0.03924651490612707}}